# Imports

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Function Definitions

## Rotation Matrices

In [ ]:
def xRot(theta):
    # Rotation matrix for the x axis
    angle = math.radians(theta)
    mat = np.array([[1, 0, 0],
                   [0, math.cos(angle), math.sin(angle)],
                   [0, -math.sin(angle), math.cos(angle)]])
    return mat

In [ ]:
def yRot(theta):
    # Rotation matrix for the y axis
    angle = math.radians(theta)
    mat = np.array([[math.cos(angle), 0, math.sin(angle)],
                   [0, 1, 0],
                   [-math.sin(angle), 0, math.cos(angle)]])
    return mat

In [ ]:
def zRot(theta):
    # Rotation matrix for the z axis
    angle = math.radians(theta)
    mat = np.array([[math.cos(angle), math.sin(angle), 0],
                   [-math.sin(angle), math.cos(angle), 0],
                   [0, 0, 1]])
    return mat

## Body Functions

In [ ]:
def bodyPos(pitch = 0, roll = 0, yaw = 0, Tx = 0, Ty = 0, Tz = 0):
    # Applies rotations in pitch, roll, and yaw and translations in x, y, and z tot he body.
    body_rot = np.matmul(zRot(yaw), yRot(roll))
    body_rot = np.matmul(body_rot, xRot(pitch));
    
    body_0 = np.matmul(inv(body_rot), np.array([[body_offset / 2, body_offset * math.sin(math.pi / 3), 0]]).T);
    body_1 = np.matmul(inv(body_rot), np.array([[body_offset, 0, 0]]).T);
    body_2 = np.matmul(inv(body_rot), np.array([[body_offset / 2, -body_offset * math.sin(math.pi / 3), 0]]).T);
    body_3 = np.matmul(inv(body_rot), np.array([[-body_offset / 2, -body_offset * math.sin(math.pi / 3), 0]]).T);
    body_4 = np.matmul(inv(body_rot), np.array([[-body_offset, 0, 0]]).T);
    body_5 = np.matmul(inv(body_rot), np.array([[-body_offset / 2, body_offset * math.sin(math.pi / 3), 0]]).T);
    
    body_model = np.concatenate((body_0.T, body_1.T, body_2.T, body_3.T, body_4.T, body_5.T, body_0.T), axis = 0)
    translation = [[Tx, Ty, Tz],
    [Tx, Ty, Tz],
    [Tx, Ty, Tz],
    [Tx, Ty, Tz],
    [Tx, Ty, Tz],
    [Tx, Ty, Tz],
    [Tx, Ty, Tz]]
    
    body_model = body_model + translation
    
    return body_model

## Leg Functions

In [ ]:
def legPos(coax_angle, femur_angle, tibia_angle, body_model, leg_num):
    # finds the positions for the coax, femur, and tibia leg segments and adds them to the body model
    coax_rot = zRot(coax_angle)
    femur_rot = np.matmul(yRot(femur_angle), coax_rot)
    tibia_rot = np.matmul(yRot(tibia_angle), femur_rot)
    
    leg_coax = np.matmul(inv(coax_rot), np.array([[coax, 0, 0]]).T) + np.array([body_model[leg_num, :]]).T
    leg_femur = np.matmul(inv(femur_rot), np.array([[femur, 0, 0]]).T) + leg_coax
    leg_tibia = np.matmul(inv(tibia_rot), np.array([[0, 0, -tibia]]).T) + leg_femur
                       
    leg_model = np.concatenate((np.array([body_model[leg_num, :]]), leg_coax.T, leg_femur.T, leg_tibia.T), axis = 0)
    return leg_model

In [ ]:
def legAngle(x, y, z):
    # finds the angles for the coax, femur, and tibia leg segments
    coax_angle = math.degrees(math.atan2(y, x));
    coax_rot = zRot(-coax_angle)
    leg_rotated = np.matmul(inv(coax_rot), np.array([[x, y, z]]).T)
    femur_angle = math.degrees(math.acos((tibia ** 2 - femur ** 2 - leg_rotated[2] ** 2 - (leg_rotated[0] - coax) ** 2) / (-2 * femur * (math.sqrt(leg_rotated[2] ** 2 + (leg_rotated[0] - coax) ** 2))))) - math.degrees(math.atan2(-leg_rotated[2], (leg_rotated[0] - coax)));
    tibia_angle = math.degrees(math.acos((leg_rotated[2] ** 2 + (leg_rotated[0] - coax) ** 2 - femur ** 2 - tibia ** 2) / (-2 * femur * tibia))) - 90;
    
    if abs(coax_angle) <= 1e-10:
        coax_angle = 0;
        
    if abs(femur_angle) <= 1e-10:
        femur_angle = 0;
    
    if abs(tibia_angle) <= 1e-10:
        tibia_angle = 0;
    
    return [coax_angle, femur_angle, tibia_angle]

## Create the starting angles of the legs on the hexapod based 

In [ ]:
def startLegPos(start_radius = 150, start_height = 20):
    start_leg0 = legAngle(start_radius * math.cos(math.pi / 3) - body_model[0, 0], start_radius * math.sin(math.pi / 3) - body_model[0, 1], - start_height - body_model[0, 2])
    start_leg1 = legAngle(start_radius - body_model[1, 0], - body_model[1, 1], - start_height - body_model[1, 2])
    start_leg2 = legAngle(start_radius * math.cos(- math.pi / 3) - body_model[2, 0], start_radius * math.sin(- math.pi / 3) - body_model[2, 1], - start_height - body_model[2, 2])
    start_leg3 = legAngle(start_radius * math.cos(- 2 * math.pi / 3) - body_model[3, 0], start_radius * math.sin(- 2 * math.pi / 3) - body_model[3, 1], - start_height - body_model[3, 2])
    start_leg4 = legAngle(- start_radius + body_model[1, 0], -body_model[4, 1], - start_height - body_model[4, 2])
    start_leg5 = legAngle(start_radius * math.cos(2 * math.pi / 3) - body_model[5, 0], start_radius * math.sin(2 * math.pi / 3) - body_model[5, 1], - start_height - body_model[5, 2])
    start_leg_combined =  np.concatenate((np.array([start_leg0]), np.array([start_leg1]), np.array([start_leg2]), np.array([start_leg3]), np.array([start_leg4]), np.array([start_leg5])), axis = 0)
    return start_leg_combined

## Plot Model

In [ ]:
def showModel():
    # plots the whole hexapod model
    fig = plt.figure()
    ax = plt.axes(projection ='3d')
    # plotting
    ax.plot3D(body_model[:, 0], body_model[:, 1], body_model[:, 2], 'green');
    for i in range(6):
        ax.plot3D(leg_model[:, 0, i], leg_model[:, 1, i], leg_model[:, 2, i], 'blue');
    ax.set_xlim(-200, 200);
    ax.set_ylim(-200, 200);
    ax.set_zlim(-100, 100);

# Hexapod Measurments

In [ ]:
# hexapod measurments
body_offset = 85 #mm
coax = 26.34 #mm
femur = 76.2 #mm
tibia = 88.32 #mm

# Body Starting Position

In [ ]:
body_model = bodyPos(0, 0, 0, 0, 0, 0);

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = plt.axes(projection ='3d')
# plotting
ax.plot3D(body_model[:, 0], body_model[:, 1], body_model[:, 2], 'green');
ax.set_xlim(-100, 100);
ax.set_ylim(-100, 100);
ax.set_zlim(-100, 100);

# Leg Starting Positions

In [ ]:
start_leg = startLegPos()
leg_model = np.empty([4, 3, 6])

for i in range(6):
    leg_model[:, :, i] = legPos(start_leg[i][0], start_leg[i][1], start_leg[i][2], body_model, i) #coax angle, femur angle, tibia angle, model of the hexapod body, leg number
    #print out the leg end positions for debugging
    print(leg_model[3,:,i])

In [ ]:
%matplotlib qt

showModel()